<a href="https://colab.research.google.com/github/ElFosco/NLP_argument_creation/blob/main/XLNet_STS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
Install dependencies

In [ ]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.5 MB/s 


Download the pretrained XLNet model and unzip

In [ ]:
# only needs to be done once
! wget https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
! unzip cased_L-24_H-1024_A-16.zip 

--2022-01-30 17:14:37--  https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.194.128, 142.250.152.128, 173.194.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1338042341 (1.2G) [application/zip]
Saving to: ‘cased_L-24_H-1024_A-16.zip’

cased_L-24_H-1024_A 100%[===================>]   1.25G   133MB/s    in 11s     

2022-01-30 17:14:47 (119 MB/s) - ‘cased_L-24_H-1024_A-16.zip’ saved [1338042341/1338042341]

Archive:  cased_L-24_H-1024_A-16.zip
   creating: xlnet_cased_L-24_H-1024_A-16/
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.index  
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.data-00000-of-00001  
  inflating: xlnet_cased_L-24_H-1024_A-16/spiece.model  
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.meta  
  inflating: xlnet_cased_

Download extract the sts-b dataset 

In [ ]:
! wget https://data.deepai.org/Stsbenchmark.zip
! unzip Stsbenchmark.zip

--2022-01-30 17:15:11--  https://data.deepai.org/Stsbenchmark.zip
Resolving data.deepai.org (data.deepai.org)... 138.201.36.183
Connecting to data.deepai.org (data.deepai.org)|138.201.36.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409703 (400K) [application/zip]
Saving to: ‘Stsbenchmark.zip’

Stsbenchmark.zip    100%[===================>] 400.10K   908KB/s    in 0.4s    

2022-01-30 17:15:14 (908 KB/s) - ‘Stsbenchmark.zip’ saved [409703/409703]

Archive:  Stsbenchmark.zip
   creating: stsbenchmark/
  inflating: stsbenchmark/readme.txt  
  inflating: stsbenchmark/sts-test.csv  
  inflating: stsbenchmark/correlation.pl  
  inflating: stsbenchmark/LICENSE.txt  
  inflating: stsbenchmark/sts-dev.csv  
  inflating: stsbenchmark/sts-train.csv  


Git clone XLNet repo for access to run_classifier and the rest of the xlnet module

In [ ]:
! git clone https://github.com/zihangdai/xlnet.git

Cloning into 'xlnet'...
remote: Enumerating objects: 122, done.
remote: Total 122 (delta 0), reused 0 (delta 0), pack-reused 122
Receiving objects: 100% (122/122), 2.92 MiB | 15.66 MiB/s, done.
Resolving deltas: 100% (59/59), done.


## Define Variables
Define all the dirs: data, xlnet scripts & pretrained model. 
If you would like to save models then you can authenticate a GCP account and use that for the OUTPUT_DIR & CHECKPOINT_DIR - you will need a large amount storage to fix these models. 

Alternatively it is easy to integrate a google drive account, checkout this guide for [I/O in colab](https://colab.research.google.com/notebooks/io.ipynb) but rememeber these will take up a large amount of storage. 


In [ ]:
SCRIPTS_DIR = 'xlnet' #@param {type:"string"}
DATA_DIR = 'stsbenchmark' #@param {type:"string"}
OUTPUT_DIR = 'proc_data/sts-b' #@param {type:"string"}
PRETRAINED_MODEL_DIR = 'xlnet_cased_L-24_H-1024_A-16' #@param {type:"string"}
CHECKPOINT_DIR = 'exp/sts-b' #@param {type:"string"}

## Run Model
This will set off the fine tuning of XLNet. There are a few things to note here:


1.   This script will train and evaluate the model
2.   This will store the results locally on colab and will be lost when you are disconnected from the runtime
3.   This uses the large version of the model (base not released presently)
4.   We are using a max seq length of 128 with a batch size of 8 please refer to the [README](https://github.com/zihangdai/xlnet#memory-issue-during-finetuning) for why this is.
5. This will take approx 4hrs to run on GPU.



In [ ]:
# to fix the dependecies issues
!pip install tensorflow-gpu==1.15.0

     |████████████████████████████████| 411.5 MB 8.4 kB/s 
     |████████████████████████████████| 503 kB 46.6 MB/s 
     |████████████████████████████████| 3.8 MB 41.0 MB/s 
     |████████████████████████████████| 50 kB 5.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=5b0e8daf91384c2eab6d7994190830e1bc79b44893687e0b1cddd84eb3374240
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succe

In [ ]:

train_command = "python xlnet/run_classifier.py \
  --do_train=True \
  --do_eval=False \
  --eval_all_ckpt=False \
  --task_name=sts-b \
  --data_dir="+DATA_DIR+" \
  --output_dir="+OUTPUT_DIR+" \
  --model_dir="+CHECKPOINT_DIR+" \
  --uncased=False \
  --spiece_model_file="+PRETRAINED_MODEL_DIR+"/spiece.model \
  --model_config_path="+PRETRAINED_MODEL_DIR+"/xlnet_config.json \
  --init_checkpoint="+PRETRAINED_MODEL_DIR+"/xlnet_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=8 \
  --eval_batch_size=8 \
  --num_hosts=1 \
  --num_core_per_host=1 \
  --learning_rate=2e-5 \
  --train_steps=4000 \
  --warmup_steps=500 \
  --save_steps=500 \
  --iterations=500"

! {train_command}





W0130 17:19:43.970555 140257370036096 module_wrapper.py:139] From xlnet/run_classifier.py:637: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0130 17:19:43.970804 140257370036096 module_wrapper.py:139] From xlnet/run_classifier.py:637: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0130 17:19:43.971073 140257370036096 module_wrapper.py:139] From xlnet/run_classifier.py:661: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.


W0130 17:19:43.971373 140257370036096 module_wrapper.py:139] From xlnet/run_classifier.py:662: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0130 17:19:44.019254 140257370036096 module_wrapper.py:139] From /content/xlnet/model_utils.py:27: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


W0130 17:19:44.019536 140257370036096 module_wrapper.py:139] From /cont

## Running & Results
These are the results that I got from running this experiment
### Params
*    --max_seq_length=128 \
*    --train_batch_size= 8 

### Times
*   Training: 1hr 11mins
*   Evaluation: 2.5hr

### Results
*  Most accurate model on final step
*  Accuracy: 0.92416, eval_loss: 0.31708


### Model

*   The trained model checkpoints can be found in 'exp/imdb'

